In [ ]:
import requests
import math

# Replace with your MapMyIndia API keys
MAPMYINDIA_CLIENT_ID = 'your_client_id'
MAPMYINDIA_CLIENT_SECRET = 'AXBHUTB-74958-NFDSNSNYMZLK-7593055'

# Function to get an access token from MapMyIndia API
def get_mapmyindia_access_token():
    url = f'https://outpost.mapmyindia.com/api/security/oauth/token'
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded'
    }
    data = {
        'grant_type': 'client_credentials',
        'client_id': MAPMYINDIA_CLIENT_ID,
        'client_secret': MAPMYINDIA_CLIENT_SECRET
    }
    response = requests.post(url, headers=headers, data=data)
    token = response.json().get('access_token')
    return token

# Function to get live vehicle tracking data (latitude, longitude)
def get_vehicle_location(vehicle_id, access_token):
    url = f'https://atlas.mapmyindia.com/api/advancedmaps/v1/track/{vehicle_id}'
    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    response = requests.get(url, headers=headers)
    data = response.json()
    return data['latitude'], data['longitude'] 

# Haversine formula: Rejects Eucledian geometry as Earth is round and adds curvature as parameter
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0  # Earth radius (constant)
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = math.sin(dlat / 2)**2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c
    return distance

# Geofence alert function
def geofence_alert(start_lat, start_lon, geofence_radius_km, access_token, vehicle_id):
    distance_travelled = 0.0
    prev_lat, prev_lon = start_lat, start_lon
    print(f"Tracking started from ({start_lat}, {start_lon})")

    while True:
        # current location
        curr_lat, curr_lon = get_vehicle_location(vehicle_id, access_token)
        
        # Calculate distance from last position
        dist = haversine(prev_lat, prev_lon, curr_lat, curr_lon)
        distance_travelled += dist

        if distance_travelled >= geofence_radius_km:
            print(f"Alert: Truck crossed {geofence_radius_km} km. Current location: ({curr_lat}, {curr_lon})")
            # Reset tracking
            distance_travelled = 0.0

        # updation of previous coordinates as current coordinates so that new values can be put in place on prev coordinates
        prev_lat, prev_lon = curr_lat, curr_lon

# mapmyindia api
access_token = get_mapmyindia_access_token()


vehicle_id = 'OD-03-MF-4456'  #need to provide validation codes to handle errors in xlsx format

# Starting point
start_lat = 20.2961  # bbsr lat
start_lon = 85.8245  # bbsr long

# Geofence radius that raises alert every 15 km
geofence_radius_km = 15

# Start geofencing and alert system
geofence_alert(start_lat, start_lon, geofence_radius_km, access_token, vehicle_id)


#Capturing real time data with our api
warehouse_coords = (20.2961, 85.8245)  # Example: Bhubaneswar Warehouse (Latitude, Longitude)
destination_coords = (19.8135, 85.8312)  # Example: Puri Destination (Latitude, Longitude)

# Simulating a truck's route with a list of GPS coordinates (latitude, longitude) between warehouse and destination
# You can replace this with actual GPS data from a dataset
truck_route_coords = [
    (20.2500, 85.8200),
    (20.2100, 85.8150),
    (20.1600, 85.8100),
    (20.1000, 85.8050),
    (20.0500, 85.8000),  # 15km alert likely around this point
    (20.0000, 85.7950),
    (19.9500, 85.7900),
    (19.9000, 85.7850),  # Another alert likely here
    (19.8500, 85.7800),
    (19.8135, 85.8312)   # Reaching Puri (Destination)
]